In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install indic
!pip install indic-nlp-library
!pip install transformers
!pip install wxconv
!pip install polyglot_tokenizer

!pip install torch_sparse
!pip install torch_geometric
!pip install torch_scatter


In [3]:
import nltk
nltk.download('punkt')
from nltk.util import ngrams

import sys

from nltk.tokenize import word_tokenize
from itertools import combinations
from collections import Counter
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize 
from nltk.tokenize import word_tokenize
import polyglot_tokenizer
from polyglot_tokenizer import Tokenizer
 
import numpy as np
import pandas as pd 
import re
import xlrd
import csv
from wxconv import WXC

import gensim
import sklearn
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 

import scipy
from scipy import spatial
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split


# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"C:\Users\ankunchu\Documents\src\indic_nlp_library"
# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"C:\Users\ankunchu\Documents\src\indic_nlp_resources"
sys.path.append(r'{}\src'.format(INDIC_NLP_LIB_HOME))# The path to the local git repo for Indic NLP library

In [ ]:
import torch 
from torch.autograd import Variable
from torch_geometric.data import Data

import transformers
from transformers import BertConfig, BertModel
from transformers import AutoTokenizer, AutoModel

In [4]:
#BERT model
# Download model and configuration from S3 and cache.

bert_model = BertModel.from_pretrained('subbareddyiiit/BERT-NLP')
config = BertConfig.from_pretrained("subbareddyiiit/BERT-NLP",output_attentions=True)  
bert_tk = AutoTokenizer.from_pretrained("subbareddyiiit/BERT-NLP")  


In [ ]:
def get_wx_sentence(sent):
  con = WXC(order='utf2wx',lang='tel')  
  wx_sentence = con.convert(sent)  
  return wx_sentence

In [ ]:
#returns a 768(for bert) size embedding 

def bert_sentence_embedding(sentence, model):
  wx_sent = get_wx_sentence(sentence)
  tokens = bert_tk.tokenize(wx_sent)
  count = 0
  for each_word in tokens:
    if (each_word=='.' or each_word=='-' or each_word=='\\' or each_word=='_' or each_word==',' or each_word=="'" or each_word=='[' or each_word==']' or each_word=='(' or each_word==')' or each_word=='*' or each_word==';' or each_word=='|'  or each_word==':'  or each_word=='-'or each_word=='?'or each_word=='!' or each_word=="\/"):
      count += 1
  if (count >= (len(tokens)/2)):
    return []
    
  sent = bert_tk.encode(sentence)
  sent = torch.tensor(sent).unsqueeze(0)
  output = model(sent)
  final_output = output.pooler_output.detach().numpy()
  return final_output[0]


In [ ]:
class Document:
  def __init__(self, doc_num):
    self.doc_num = doc_num
      
  def add_clustermap(self, labels, k):
    temp = {}
    for i in range(k):
      temp[i] = []
    for i,each_label in enumerate(labels):
      temp[each_label].append(i)
    self.cluster_map = temp
  

In [ ]:
#pre-processing step 

def pre_process_sentences(sentences):
  sentences = sentences.replace('\u200c','')
  sentences = sentences.replace('\\u200c','')
  sentences = sentences.replace('\\u200d','')
  sentences = sentences.split('[')[1]
  sentences = sentences.split(']')[0]
  sentences = re.split("', '|', \"|\", '|\", \"",  sentences)
  n = len(sentences)
  if (sentences[0][0] == "'"):
    sentences[0] = sentences[0].split("'")[1]
  elif (sentences[0][0] == '"'):
    sentences[0] = sentences[0].split('"')[1]
  else:
    print("character = ", sentences[0][0])
  
  m = len(sentences[n-1])
  if (sentences[n-1][m-1] == "'"):
    sentences[n-1] = sentences[n-1].split("'")[0]    
  elif (sentences[n-1][m-1] == '"'):
    sentences[n-1] = sentences[n-1].split('"')[0] 
  else:
    print("ending character = ", sentences[n-1][m-1])
  return sentences


def pre_process_highlights(sentences):
  sentences = sentences.replace('\u200c','')
  sentences = sentences.replace('\\u200c','')
  sentences = sentences.replace('\\u200d','')
  sentences = sentences.split('[')[1]
  sentences = sentences.split(']')[0]
  sentences = sentences.split("', '")
  sentences[0] = sentences[0].split("'")[1]
  n = len(sentences)
  sentences[n-1] = sentences[n-1].split("'")[0]
  return sentences

In [ ]:
def get_clusters( embeds, k):
  clustering = SpectralClustering(n_clusters=k, assign_labels="discretize", random_state=0).fit(embeds)
  return clustering.labels_


In [5]:
#dataset you want to try upon
final_data = open('/content/drive/My Drive/summarization/samyam_data/data/final_final_samyamdata.csv')

count = 0
data_reader = csv.reader(final_data)
k = 3   #no.of clusters
emb_dim = 768     

all_documents = []
count=0

for row in data_reader:
  # try:
    if count ==0:
      count=1
      continue
    doc = Document(row[1])
    highlights = pre_process_highlights(row[4])
    sentences = pre_process_sentences(row[7])
    setattr(doc, "highlights", highlights)
    setattr(doc, "raw_sentences", sentences)
    
    embeds = []
    for sent in sentences:
      temp = bert_sentence_embedding(sent, bert_model)
      embeds.append(temp)
    setattr(doc, "sent_embeds", embeds)
    if (len(embeds)!=len(sentences)):
      print("sentence improper = ", sentences)
      raise ValueError
    all_documents.append(doc)  
    count += 1


#Getting document representation

In [6]:
# getting document embeddings by summation of sentence representations

def get_document_embedding(all_documents):       
  doc_embeds = []
  for each_doc in all_documents:
    doc_embedding = np.zeros(emb_dim)
    for sent in each_doc.sent_embeds:
      doc_embedding = np.add(doc_embedding, sent)
    doc_embedding = doc_embedding/len(each_doc.sent_embeds)
    doc_embeds.append(doc_embedding)
  return doc_embeds

doc_embeds1 = get_document_embedding(all_documents)


#Padding sentences

In [ ]:
def get_max_length(all_documents):
  max_l = 0
  for each_doc in all_documents:
    if max_l < len(each_doc.sent_embeds):
      max_l = len(each_doc.sent_embeds)
  print(max_l)
  return max_l

In [ ]:
def pad_sentences(max_length):
  for i in range(len(all_documents)):
    req = max_length -len(all_documents[i].sent_embeds)
    if req >0:
      added = np.zeros((req, emb_dim))
      all_documents[i].sent_embeds.extend(added)
      add_labels = [-1 for i in range(req)]

In [9]:
max_num_of_sent = get_max_length(all_documents)
pad_sentences(max_num_of_sent)

# Document Graph Construction 


In [ ]:
import sklearn
import scipy
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

def get_cosine_similarity(sent_a, sent_b):
  return (1 - spatial.distance.cosine(sent_a, sent_b))

def doc_edge_index(x, threshold):
  n = len(x)
  pos_edge_index_r = []
  pos_edge_index_c = []
  neg_edge_index_r = []
  neg_edge_index_c = []
  for i in range(n):
    for j in range(i+1,n):
      try:
        sent1 = x[i]
        sent2 = x[j]
        score = get_cosine_similarity( sent1, sent2)
      except:
        print("error = ", sent1, sent2)
        raise ValueError
      if (score > threshold):
        pos_edge_index_r.append(i)
        pos_edge_index_c.append(j)
        pos_edge_index_r.append(j)
        pos_edge_index_c.append(i)
      else:
        neg_edge_index_r.append(i)
        neg_edge_index_c.append(j)
        neg_edge_index_r.append(j)
        neg_edge_index_c.append(i)
  pos_edge_index = torch.tensor([pos_edge_index_r, pos_edge_index_c])
  neg_edge_index = torch.tensor([neg_edge_index_r, neg_edge_index_c])
  return pos_edge_index, neg_edge_index


In [ ]:
#edge index for document noded graph

doc_threshold = 0.997
doc_embeds_pos_edge_index, doc_embeds_neg_edge_index = doc_edge_index(doc_embeds1, doc_threshold)


# **Document Encoding**
 > Document graph constructed is passed through GAE_doc to obtain document representations.






In [ ]:
from torch_geometric.nn import GAE
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [ ]:

class Document_Encoder_GAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Document_Encoder_GAE, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


In [ ]:
import shutil
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    f_path = checkpoint_path
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)


In [7]:
doc_gae_model = GAE(Document_Encoder_GAE(768, 256))
doc_optimizer = torch.optim.RMSprop(doc_gae_model.parameters() , lr=1e-4, weight_decay=5e-4)
print("optimizer = ", doc_optimizer)

#number of epochs
train_loss_min = np.Inf
for t in range(40):
    print(" starting epoch = ", t)
    doc_gae_model.train()
    doc_optimizer.zero_grad()
    doc_em = torch.FloatTensor(doc_embeds1)
    output = doc_gae_model.encode(doc_em, doc_embeds_pos_edge_index)
    train_loss = doc_gae_model.recon_loss(output, doc_embeds_pos_edge_index)
    train_loss.backward()
    doc_optimizer.step()
    doc_gae_model.eval()

    checkpoint = {
            'epoch': t + 1,
            'valid_loss_min': train_loss,
            'state_dict1': doc_gae_model.state_dict(),
            'optimizer': doc_optimizer.state_dict(),}

    save_ckp(checkpoint, False, 'doc_joint_checkpoint1.pt', 'doc_joint_best_model1.pt')
    print("loss = ", train_loss)
    if train_loss <= train_loss_min:
      print(' loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(train_loss_min,train_loss))
      save_ckp(checkpoint, True,  'doc_joint_checkpoint2.pt',  'doc_joint_best_model2.pt')
      train_loss_min = train_loss


In [10]:
final_doc_embeds = doc_gae_model.encode(torch.FloatTensor(doc_embeds1), doc_embeds_pos_edge_index )
print(final_doc_embeds.shape)

# **Sentence Encoding and Summary Generation**

# Sentence Graph construction

In [ ]:
def sent_edge_index(x, threshold):
  n = len(x.raw_sentences)
  pos_edge_index_r = []
  pos_edge_index_c = []
  neg_edge_index_r = []
  neg_edge_index_c = []
  for i in range(n):
    for j in range(i+1,n):
      try:
        sent1 = x.sent_embeds[i]
        sent2 = x.sent_embeds[j]
        score = get_cosine_similarity( sent1, sent2)
      except:
        print("error = ", x.sent_embeds.shape)
        raise ValueError
      if (score > threshold):
        pos_edge_index_r.append(i)
        pos_edge_index_c.append(j)
        pos_edge_index_r.append(j)
        pos_edge_index_c.append(i)
      else:
        neg_edge_index_r.append(i)
        neg_edge_index_c.append(j)
        neg_edge_index_r.append(j)
        neg_edge_index_c.append(i)
  pos_edge_index = torch.tensor([pos_edge_index_r, pos_edge_index_c])
  neg_edge_index = torch.tensor([neg_edge_index_r, neg_edge_index_c])
  return pos_edge_index, neg_edge_index


In [ ]:
sent_threshold = 0.99

for i,each_doc in enumerate(all_documents):
  pos_edge_index, neg_edge_index =sent_edge_index(each_doc, sent_threshold)
  setattr(each_doc, "pos_edge_index", pos_edge_index)
  setattr(each_doc, "neg_edge_index", neg_edge_index)

#Sentence Encoding

In [ ]:
class Encoder_GAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder_GAE, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

#Clustering and Cluster Embedding

In [ ]:
class RNN_model(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim):
    super(RNN_model, self).__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim
    self.input_dim = input_dim
    
    self.rnn = torch.nn.GRU(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)

  
  def forward(self, x, sent_embeds):
    #modify the document to get the proper input
    final_input = []
    lengths = []

    len_sent = len(x.raw_sentences)
    copy_sent = sent_embeds[:len_sent].clone().detach()
    labels = get_clusters(copy_sent, k)
    x.add_clustermap(labels, k)
    req = max_num_of_sent -len(x.raw_sentences)
    if req >0:
      add_labels = [-1 for i in range(req)]
      labels = np.append(labels, add_labels)
    setattr(x, "labels", labels)

    map = x.cluster_map

    for key,value in map.items():
      temp = torch.empty((len(value),self.input_dim,))
      for l,each_val in enumerate(value):
        temp[l] = sent_embeds[each_val]
      if (len(value) == 0):
        lengths.append(1)
        final_input.append(torch.zeros(self.input_dim))
      else:
        final_input.append(temp)
        lengths.append(len(value))

    b = torch.nn.utils.rnn.pad_sequence(final_input, batch_first=True)    
    my_packed_seq = torch.nn.utils.rnn.pack_padded_sequence(b, lengths, batch_first=True, enforce_sorted=False)

    # Initialize hidden state with zeros  (change initialization)
    h0 = torch.zeros(self.layer_dim, len(final_input), self.hidden_dim)
    out, hn = self.rnn(my_packed_seq, h0)
    unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)

    #get proper document wise cluster embeddings
    final_clu_embeds = torch.zeros((k,self.hidden_dim))

    for j in range(k):
      final_clu_embeds[j] = unpacked[j,unpacked_len[j]-1,:]
    return final_clu_embeds

#Summary generation
The actual model in order to find the top k sentences in the document. 

Gets the relevance and position scores of each sentence and then finds the top k sentences


In [ ]:
class Summary_model(torch.nn.Module):
  def __init__(self,Dim, H1):
    super(Summary_model, self).__init__()
    self.linear1 = torch.nn.utils.weight_norm(torch.nn.Linear(Dim, H1), name='weight')
    

  def get_salience_scores(self, doc_scores, labels):
    clu_scores = torch.zeros(k+1)
    new_scores = torch.zeros(len(doc_scores))
    for i in range(len(doc_scores)):
      clus_num = labels[i]
      clu_scores[clus_num+1] = torch.add(clu_scores[clus_num+1], doc_scores[i])
    clu_scores[0] = 0         #padded sentences

    for i in range(len(doc_scores)):
      clus_num = labels[i]
      if (clus_num+1==0) :
        new_scores[i] = 0.0
      else:
        new_scores[i] = torch.div(doc_scores[i],(clu_scores[clus_num+1]))
    return new_scores
  

  def append_cluster_embeds(self, labels, sentences, clu_embed):
    size_em = 128     #cluster embedding size (hidden dimension)
    size = 256+size_em  #gcn output sentence embedding     
    append_embeds = torch.zeros( (sentences.shape[0], size ) )
    
    for i,each_sent in enumerate(sentences):
      l = labels[i]
      if (l == -1):
        cl_em = torch.zeros(size_em)
        temp = torch.cat(( torch.tensor(sentences[i]), torch.zeros(size_em, requires_grad=True)  ), 0)
      else:
        temp = torch.cat((torch.tensor(sentences[i]), clu_embed[l]), 0)
      append_embeds[i] = temp
    return append_embeds
  
  def get_position_score(self,x):
    pos_scores = torch.zeros(len(x.sent_embeds))
    N = len(x.raw_sentences)
    for i in range(N):
      value = (i+1)/(N ** (1./3))
      pos_scores[i] = max(0.5, math.exp(-value))
    return pos_scores

  
  def forward(self, x, cluster_embeds, sent_embeds):
    #access the embeddings
    final_embeds = self.append_cluster_embeds(x.labels, sent_embeds, cluster_embeds)
    scores = self.linear1(final_embeds)
    scores = torch.tanh(scores)
    sal_scores = self.get_salience_scores(scores, x.labels)
    pos_scores = self.get_position_score(x)
    final_scores = torch.add(0.4*sal_scores, 0.6*pos_scores)
    return final_scores

In [ ]:
class Ensemble_model(torch.nn.Module):
  def __init__(self, model1,model2):
    super(Ensemble_model, self).__init__()
    self.model1 = model1
    self.model2 = model2

  def forward(self, x, sent_embeds):
    x1 = self.model1(x, sent_embeds)
    x2 = self.model2(x, x1, sent_embeds) 
    return x2

#Loss function

In [ ]:
import torch.nn.functional as F
#Contrastive Loss function

class ContrastiveLoss(torch.nn.Module):

    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.eps = 1e-9

    def forward(self, output1, output2, target, size_average=True):
        distances = (output2 - output1).pow(2).sum(0)  # squared distances
        losses = 0.5 * (float(target) * distances + float(1 + -1 * target) * F.relu(self.margin - (distances + self.eps).sqrt()).pow(2))
        return losses.mean() if size_average else losses.sum()


def logcosh(y_t, y_prime_t):
  ey_t = y_t - y_prime_t
  return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))


In [ ]:
import torch.nn.functional as F

def my_loss_function(y_pred, y_true, x, sent_embeds):

  summary = torch.zeros(256, requires_grad=True)
  final_scores = torch.zeros((len(y_pred),257))
 
  for i in range(len(y_pred)):
    final_scores[i][0] = y_pred[i]
    final_scores[i][1:] = torch.tensor(sent_embeds[i], requires_grad=True)  
  
  sorted, indices = torch.sort(final_scores, 0, descending=True)
  val = 0
  for i in range(top_k_sent):
    val = val + sorted[i,0]

  for i in range(top_k_sent):
    summary = torch.add(summary, torch.mul(sorted[i, 1:], sorted[i,0]) ) 
  num = top_k_sent*val
  summary = torch.div(summary,num)
  con_loss = ContrastiveLoss(0.5)
  loss1 = con_loss(y_true,summary, 1.0 )
  return loss1


#Training the model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_documents, final_doc_embeds.detach(), test_size=0.01)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import math

Dim = 768  
top_k_sent = 3
input_dim = Dim   
hidden_dim = 128 #cluster dimension  
layer_dim = 2
gcn_out_dim = 256 #encoder bottle neck expected dimension 

#with gae
gaemodel = GAE(Encoder_GAE(Dim, gcn_out_dim))
model1 = RNN_model(gcn_out_dim, hidden_dim, layer_dim)
model2 = Summary_model(gcn_out_dim+hidden_dim, 1)
final_model = Ensemble_model( model1, model2)
all_params = list(final_model.parameters()) + list(gaemodel.parameters())
optimizer = torch.optim.RMSprop(all_params, lr=5e-4, weight_decay=5e-4)

print("optimizer = ", optimizer)

#number of epochs

train_loss_min = np.Inf
for t in range(40):
    print(" starting epoch = ", t)
    train_loss = 0.0
    final_model.train()

    for i in range(len(X_train)):
      try:
        optimizer.zero_grad()
        sentences_em = torch.FloatTensor(X_train[i].sent_embeds)
        output = gaemodel.encode(sentences_em, X_train[i].pos_edge_index)
        recon_loss = gaemodel.recon_loss(output, X_train[i].pos_edge_index)
        y_pred = final_model( X_train[i], output )
        trail_doc_embeds = np.zeros( (len(all_documents),gcn_out_dim ) )
        loss = my_loss_function( y_pred, y_train[i] , X_train[i], output)  
        Joint_loss = loss+recon_loss
        Joint_loss.backward()
        optimizer.step()
        train_loss = train_loss + ( (1 / (i + 1)) * (Joint_loss.data - train_loss))
      except:
        print("error here = ",i)

    final_model.eval()

    checkpoint = {
            'epoch': t + 1,
            'valid_loss_min': train_loss,
            'state_dict1': final_model.state_dict(),
            'state_dict2':gaemodel.state_dict(),
            'optimizer': optimizer.state_dict(),}

    save_ckp(checkpoint, False, 'joint_checkpoint2.pt', 'joint_best_model2.pt')
    print("loss = ", train_loss)
    if train_loss <= train_loss_min:
      print(' loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(train_loss_min,train_loss))
      save_ckp(checkpoint, True,  'joint_checkpoint2.pt',  'joint_best_model2.pt')
      train_loss_min = train_loss